# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json
from pprint import pprint

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

In [3]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

613

In [41]:
df = pd.DataFrame(cities)
df["City"]=cities
# df["Lat"] = lats
# df["Lng"] = lngs
df.drop([0])
df

,0,City
0,butaritari,butaritari
1,georgetown,georgetown
2,luanda,luanda
3,new norfolk,new norfolk
4,hobart,hobart
...,...,...
608,coquimbo,coquimbo
609,jiuquan,jiuquan
610,almaty,almaty
611,jalu,jalu


In [32]:
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"
weather_url = f"{url}appid={weather_api_key}&units={units}&q="

city_short_df = df.head(10)
city_short_df
lat = []
lng = []
temps = []
humidity = []
cloudiness = []
windspeed = []

# looping through test cities:
for city in city_short:
    response = requests.get(weather_url + city).json()
    lat.append(response["coord"]["lat"])
    lng.append(response["coord"]["lon"])
    temps.append(response["main"]["temp_max"])
    humidity.append(response["main"]["humidity"])
    cloudiness.append(response["weather"]["clouds"])
    windspeed.append(response["wind"]["speed"])
    

print("processing {index} city | {city}")


TypeError: can only concatenate str (not "int") to str

In [ ]:
temps = cities_df["Max Temp"]
humidity = cities_df["Humidity"]
cloudiness = cities_df["Cloudiness"]
lat = cities_df["Lat"]
windspeed = cities_df["Wind Speed"]

print(north_lat)

In [ ]:
# North Data
n_temps = north_df["Max Temp"]
n_humidity = north_df["Humidity"]
n_cloudiness = north_df["Cloudiness"]
n_windspeed = north_df["Wind Speed"]
n_lat = north_df["Lat"]

# South Data
s_temps = south_df["Max Temp"]
s_humidity = south_df["Humidity"]
s_cloudiness = south_df["Cloudiness"]
s_windspeed = south_df["Wind Speed"]
s_lat = south_df["Lat"]

In [ ]:
plt.scatter(temps, lat)
plt.title("Temp Vs Latitude")
plt.xlabel("Temperature")
plt.ylabel("Latitude")

print("As the latitude reaches the Equator, the tempurature gets warmer")

In [ ]:
plt.scatter(humidity, lat)
plt.title("Humidity Vs Latitude")
plt.xlabel("Humidity")
plt.ylabel("Latitude")

In [ ]:
plt.scatter(cloudiness, lat)
plt.title("Cloudiness Vs Latitude")
plt.xlabel("Cloudiness %")
plt.ylabel("Latitude")

In [ ]:
plt.scatter(windspeed, lat)
plt.title("WindSpeed Vs Latitude")
plt.xlabel("Windspeed (mph)")
plt.ylabel("Latitude")

In [ ]:
# Temperature (F) vs. Latitude
# Northern Hem
plt.subplot(2,2.5,1)
plt.scatter(n_temps, north_lat)
plt.title("Temp Vs Latitude- Northern Hem")


# # Southern Hem
plt.subplot(2,3,3)
plt.scatter(s_temps, south_lat)
plt.title("Temp Vs Latitude- Southern Hem")



In [ ]:
north_df = cities_df.loc[cities_df["Lat"] >0,:]
# north_df
south_df = cities_df.loc[cities_df["Lat"] <0,:]
north_df

In [ ]:
# Humidity vs. Latitude
# Northern Hem
plt.subplot(2,2.5,1)
plt.scatter(n_humidity, north_lat)
plt.title("Temp Vs Latitude- Northern Hem")


# # Southern Hem
plt.subplot(2,3,3)
plt.scatter(s_humidity, south_lat)
plt.title("Temp Vs Latitude- Southern Hem")

In [ ]:
# Cloudiness vs. Latitude
# Northern Hem
plt.subplot(2,2.5,1)
plt.scatter(n_cloudiness, north_lat)
plt.title("Temp Vs Latitude- Northern Hem")


# # Southern Hem
plt.subplot(2,3,3)
plt.scatter(s_cloudiness, south_lat)
plt.title("Temp Vs Latitude- Southern Hem")

In [ ]:
# Wind Speed vs. Latitude
# Northern Hem
plt.subplot(2,2.5,1)
plt.scatter(n_windspeed, north_lat)
plt.title("Temp Vs Latitude- Northern Hem")


# # Southern Hem
plt.subplot(2,3,3)
plt.scatter(s_windspeed, south_lat)
plt.title("Temp Vs Latitude- Southern Hem")

## Generate Cities List

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression